## Part3
Using the models.ldamodel module from the gensim library, run topic modeling over the corpus. Explore different numbers of topics (varying from 5 to 50), and settle for the parameter which returns topics that you consider to be meaningful at first sight.

In [ ]:
# imports
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from gensim import corpora, models, utils
from nltk.stem import WordNetLemmatizer


In [ ]:
data = pd.read_csv('hillary-clinton-emails/Emails.csv', index_col=0).dropna()
texts = pd.concat((data.ExtractedBodyText ,data.ExtractedSubject), axis=1)


In [ ]:
sw = stopwords.words('english') + ['re', 'fw', 'fvv', 'fwd']

In [ ]:
def fil(row):
    t = utils.simple_preprocess(row.ExtractedSubject)
    filt = list(filter(lambda x: x not in sw, t))
    return ' '.join(filt)
texts['ExtractedSubject'] = texts.apply(fil, axis=1)

In [ ]:
texts.groupby(by='ExtractedSubject', as_index=False).apply(lambda x: (x + ' ').sum())

In [ ]:
texts.ExtractedBodyText.fillna('',inplace=True)
texts.ExtractedSubject.fillna('',inplace=True)
texts['SubjectBody'] = texts.ExtractedBodyText +  ' ' + texts.ExtractedSubject
mails = texts.SubjectBody
tokens = []
for text in mails:
    t = utils.simple_preprocess(text)
    filt = list(filter(lambda x: x not in sw, t))
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(x) for x in filt]
    tokens.append(lemmatized)

dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]

In [ ]:
lda = models.LdaModel(corpus, num_topics=15, id2word = dictionary)
lda.print_topics(lda.num_topics)